In [23]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher

In [24]:
def get_text_from_csv(fn): # get ad's text from csv file, return a dataframe with text, ad_id,ad_url, and report_url
    df = pd.read_csv(fn)
    df = df[df['ad_type'] == 'Text'].reset_index(drop = True) # only get text ads
    df_url = df[['ad_id','ad_url']].reset_index(drop = True) # get the urls
    urls = df_url['ad_url'].to_list() # create a list so we could get the report_urls
    report_urls = []
    for url in urls:
        entity_id = url.split('/')[-3]
        creative_id = url.split('/')[-1]
        report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
        report_urls.append(report_url)
    df_report = pd.DataFrame(report_urls).rename(columns = {0:'report_url'})
    ad_text = [] # get texts from reports
    for report_url in report_urls:
        response = requests.get(report_url)
        text = response.text.split('"]')[0].split('[')[-1]
        ad_text.append(text)  
    df_text = pd.DataFrame(ad_text).rename(columns = {0:'text'})
    df_new = df[df['ad_type']=='Text'][['ad_id','ad_url']].reset_index(drop = True) 
    df = pd.concat([df_new,df_report,df_text],axis=1)
    return df 

In [25]:
google_df = get_text_from_csv('GoogleAds/texas.csv')
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       148 non-null    object
 1   ad_url      148 non-null    object
 2   report_url  148 non-null    object
 3   text        148 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


In [26]:
google_df.head()

,ad_id,ad_url,report_url,text
0,CR101029003875319808,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR101029003875319808,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR101029003875319808&hl=en,"""Election Day: MARCH 1"",""Vote for a Brighter Future !"
1,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada."
2,CR286730195614826496,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR286730195614826496,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR286730195614826496&hl=en,"""Let's make voting simpler, more convenient, and more accessible"",""Vote Kurt for Clerk in the March 1 Democratic Primary!"
3,CR212564562908545024,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR212564562908545024,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR212564562908545024&hl=en,"""Texas's 29th Congressional District"
4,CR493844926061084672,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR493844926061084672,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR493844926061084672&hl=en,"""Let's make voting simpler, more convenient, and more accessible"",""Vote Kurt for Clerk in the March 1 Democratic Primary!"


In [27]:
# read facebook ads
facebook_df = pd.read_csv('FacebookAds/Texas.csv')
# only keep text column
facebook_df.drop(columns = ['Unnamed: 0','byline'],inplace = True)
pd.set_option('display.max_colwidth', None)
facebook_df

,text
0,"Elon Musk's 44 billion dollar cash offer for Twitter is accepted, so a new day dawns for communication in the public square ""for the future of humanity."" Now as one Republican officeholder that really started on Twitter after the former CEO Dorsey cut off ads for political accounts, I stayed on tweeting conservative public policy solutions to Texas' challenges! I wasn't going to give up on the public square discussion and slowly doubled my audience with good content @teambettencourt. So, I wish Elon Musk well in his already announced reforms. This Reuters article covers that and more, and It's well worth the read at:\nhttps://www.reuters.com/technology/exclusive-twitter-set-accept-musks-best-final-offer-sources-2022-04-25/\n\nThe best public policy stands out in any public debate, and Musk is smart enough to realize that standing up for free speech pays tremendous dividends to all those that participate. That's why he's not worried about the economics of his purchase publicly because if he does his job, he will exceed expectations again like he did at PayPal, Tesla, and SpaceX!!"
1,"🚩 WTF: First it was Texas. And at least a DOZEN other states are at risk of passing a bill similar to Texas's extreme six-week abortion ban.\n\nThese cruel bans disproportionately harm Black & Latinx people, people with low incomes, and people in rural areas, who already face immense barriers to health care access. It's up to us to stop it >>"
2,It is essential that Dems nominate our STRONGEST advocate to win in November. Mihaela Plesa is the only candidate in this race that has the experience to deliver results on day one!
3,NaN
4,"Hey Bob Moore! What say you debate me?\n\nI challenge you to a debate on Facebook Live! this coming Friday, April 29th at 7pm.\n\n(Folks, if you’d like to see this debate, please share this post everywhere! 😊)"
...,...
28536,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28537,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28538,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28539,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."


In [28]:
# concat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text

,ad_id,ad_url,report_url,text
0,CR101029003875319808,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR101029003875319808,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR101029003875319808&hl=en,"""Election Day: MARCH 1"",""Vote for a Brighter Future !"
1,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada."
2,CR286730195614826496,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR286730195614826496,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR286730195614826496&hl=en,"""Let's make voting simpler, more convenient, and more accessible"",""Vote Kurt for Clerk in the March 1 Democratic Primary!"
3,CR212564562908545024,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR212564562908545024,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR212564562908545024&hl=en,"""Texas's 29th Congressional District"
4,CR493844926061084672,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR493844926061084672,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR493844926061084672&hl=en,"""Let's make voting simpler, more convenient, and more accessible"",""Vote Kurt for Clerk in the March 1 Democratic Primary!"
...,...,...,...,...
28536,NaN,NaN,NaN,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28537,NaN,NaN,NaN,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28538,NaN,NaN,NaN,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."
28539,NaN,NaN,NaN,"As Southlake Mayor, I lowered local property taxes NINE TIMES. I will continue standing strong for homeowners as our next Representative for HD93."


In [29]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

16


In [30]:
# function that filter the urls and symbols in the text 
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x) # filter symbols
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split()] 
    
    return ' '.join(ls)

In [31]:
df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
df_text.head()

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_51545/138548313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))


,ad_id,ad_url,report_url,text
0,CR101029003875319808,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR101029003875319808,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR101029003875319808&hl=en,election day march 1 vote for a brighter future
1,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada
2,CR286730195614826496,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR286730195614826496,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR286730195614826496&hl=en,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary
3,CR212564562908545024,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR212564562908545024,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR212564562908545024&hl=en,texas s 29th congressional district
4,CR493844926061084672,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR493844926061084672,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR493844926061084672&hl=en,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary


In [32]:
# remove duplicated rows
df_text.drop_duplicates(subset = 'text',keep = 'first',inplace = True)
df_text.reset_index(inplace = True,drop = True)
df_text.head()

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_51545/3853980377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text.drop_duplicates(subset = 'text',keep = 'first',inplace = True)


,ad_id,ad_url,report_url,text
0,CR101029003875319808,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR101029003875319808,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR101029003875319808&hl=en,election day march 1 vote for a brighter future
1,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada
2,CR286730195614826496,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR286730195614826496,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR286730195614826496&hl=en,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary
3,CR212564562908545024,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR212564562908545024,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR212564562908545024&hl=en,texas s 29th congressional district
4,CR554675269428838400,https://transparencyreport.google.com/political-ads/library/advertiser/AR137351233059946496/creative/CR554675269428838400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR137351233059946496&creative_id=CR554675269428838400&hl=en,vote for brandon williams republican for ny 22 patriot not a politician


In [33]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       87 non-null     object
 1   ad_url      87 non-null     object
 2   report_url  87 non-null     object
 3   text        384 non-null    object
dtypes: object(4)
memory usage: 12.1+ KB


In [44]:
df_text.to_csv('ad_tx.csv')

In [34]:
# import the Lexicon
with open ('lexicon.json') as f1:
    dic1 = json.load(f1)
# check the keys
dic1.keys()

dict_keys(['covid', 'economic', 'education', 'environment', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social and cultural', 'social programs', 'donation'])

In [35]:
# import en_core_web
#This only happens to Ying that she couldn't import 'en_core_web_sm' in Jupyter Notebook, so she imports it with the full path
nlp = spacy.load("/usr/local/Cellar/jupyterlab/3.2.9/libexec/lib/python3.9/site-packages/en_core_web_lg/en_core_web_lg-3.2.0/")

In [36]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        matcher = PhraseMatcher(nlp.vocab)
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", patterns) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')

In [37]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [38]:
df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
df_text['m_label'] = df_text['m_label'].apply(lambda x: 'no topic' if x=='' else x)
df_text

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_51545/4081148879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_51545/4081148879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_51545/4081148879.py:3: SettingWithCopyWa

,ad_id,ad_url,report_url,text,words,m_label
0,CR101029003875319808,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR101029003875319808,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR101029003875319808&hl=en,election day march 1 vote for a brighter future,no words,no topic
1,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,no words,no topic
2,CR286730195614826496,https://transparencyreport.google.com/political-ads/library/advertiser/AR234519885969883136/creative/CR286730195614826496,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR234519885969883136&creative_id=CR286730195614826496&hl=en,let s make voting simpler more convenient and more accessible vote kurt for clerk in the march 1 democratic primary,no words,no topic
3,CR212564562908545024,https://transparencyreport.google.com/political-ads/library/advertiser/AR63171272342765568/creative/CR212564562908545024,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR63171272342765568&creative_id=CR212564562908545024&hl=en,texas s 29th congressional district,no words,no topic
4,CR554675269428838400,https://transparencyreport.google.com/political-ads/library/advertiser/AR137351233059946496/creative/CR554675269428838400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR137351233059946496&creative_id=CR554675269428838400&hl=en,vote for brandon williams republican for ny 22 patriot not a politician,no words,no topic
...,...,...,...,...,...,...
379,NaN,NaN,NaN,transportation roads and traffic are the top issues for the folks in hd93,transportation,economic
380,NaN,NaN,NaN,i championed school safety and put a school resource officer in every carroll isd school to keep students safe defund the police radicals tried to remove those officers but i fought back and won i refuse to back down when the safety of our families and students is on the line i m a proven conservative fighter and i ll get the job done for us here in hd93,"police,officers,radicals,school","education,safety,social and cultural"
381,NaN,NaN,NaN,as the former southlake mayor i have firsthand experience fighting back against the critical race theory agenda that pits neighbor against neighbor the american dream is open and accessible to all who are willing to work hard that starts with investing in a great education for our students keeping politicized curriculums out of the classroom and ensuring our schools stay open for in person learning click and stand with me for a better education system today,education,education
382,NaN,NaN,NaN,laura hill has fought back against critical race theory from day 1 and she has the battle scars to prove it parents should have the final say in their children s education and we have spoken no crt and no liberal political agendas in our schools,education,education


In [39]:
 df_text.to_csv('ad_tx.csv')

In [40]:
def count_tag(df_text):
    df_tag = df_text['m_label'].value_counts().rename_axis('topics').reset_index(name='counts')
    df_tag = df_tag.assign(single_topic=df_tag['topics'].str.split(',')).explode('single_topic').reset_index(drop = True)
    df_tag = df_tag.groupby('single_topic').sum().reset_index().sort_values(by = 'counts', ascending = False)
    return df_tag

In [41]:
count_tag(df_text)

,single_topic,counts
10,no topic,175
2,economic,63
8,immigration,41
11,safety,39
12,social and cultural,37
3,education,35
1,donation,24
7,health,24
6,governance,22
4,environment,20


In [42]:
def count_word(df_text):
    df_words = df_text['words'].value_counts().rename_axis('words').reset_index(name='counts')
    df_words = df_words.assign(single_word=df_words['words'].str.split(',')).explode('single_word').reset_index(drop = True)
    df_words = df_words.groupby('single_word').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_words.reset_index(drop = True, inplace = True)
    return df_words

In [43]:
count_word(df_text)

,single_word,counts
0,no words,175
1,border,30
2,taxes,21
3,tax,18
4,health,15
...,...,...
198,gay,1
199,graduates,1
200,gun,1
201,illegal immigration,1
